In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk.classify.util

In [2]:
data = pd.read_csv('Amazon.csv')
print(data.shape)
data.head(5)

(455000, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [20]:
# features from Amazon.csv to add to feature set

#features related to text body of review
data['reviewLen'] = data['Text'].str.len()
data['hasEP'] = data['Text'].str.contains('!')
data['hasQM'] = data['Text'].str.contains('\?')
data['hasAST'] = data['Text'].str.contains('\*')
data['hasDS'] = data['Text'].str.contains('$')
data['hasSC'] = data['Text'].str.contains(';')
data['hasAT'] = data['Text'].str.contains('@')
data['bodyNum'] = data['Text'].str.contains('\d')

data['EPcount'] = data['Text'].str.count('!')
data['QMcount'] = data['Text'].str.count('\?')
data['ASTcount'] = data['Text'].str.count('\*')
data['DScount'] = data['Text'].str.count('$')
data['SCcount'] = data['Text'].str.count(';')
data['ATcount'] = data['Text'].str.count('@')
data['NUMcount'] = data['Text'].str.count('\d')

#features related to profile name 
data['nameLen'] = data['ProfileName'].str.len()
data['nameNum'] = data['ProfileName'].str.contains('\d')
data['nameNumCount'] = data['ProfileName'].str.count('\d')
data['nameQUOTE'] = data['ProfileName'].str.contains('"')
data['nameSPACE'] = data['ProfileName'].str.contains(' ')

#time? sooner the better? 
data['first'] = data['Time'].rank(ascending=1)

data.head(5)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,...,EPcount,QMcount,ASTcount,DScount,SCcount,NUMcount,nameNumCount,hasAT,ATcount,first
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,...,0,0,0,1,0,0,0,False,0,166622.5
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,...,0,0,0,1,0,0,0,False,0,443838.5
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,...,6,0,0,1,1,0,0,False,0,321078.5
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,...,1,0,0,1,0,0,1,False,0,76162.0
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,...,1,0,0,1,0,0,0,False,0,344549.5


In [ ]:
#turns colum into vector - here columns 7 and 13
XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)
XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
XhasEP = data.iloc[:, 14].values.reshape(data.shape[0], 1)
XhasDS = data.iloc[:, 15].values.reshape(data.shape[0], 1)
XhasSC = data.iloc[:, 16].values.reshape(data.shape[0], 1)
XhasQM = data.iloc[:, 17].values.reshape(data.shape[0], 1)
XhasAST = data.iloc[:, 18].values.reshape(data.shape[0], 1)
    

Xtoadd = np.concatenate((XScore, XreviewLen, XhasEP, XhasDS, XhasSC, XhasQM, XhasAST), axis=1)

In [ ]:
# report on training and test sets
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training tet:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [ ]:
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
X = hv.transform(data.Text)

#if you restrict it to 2 to the power of 17 features is a few extra, way to restrict number of features created 

In [ ]:
# convert additional features to sparse matrix and concatenate onto the bag of words sparse matrix
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
Xfinal = hstack([X, XtoaddSparse])
X = csr_matrix(Xfinal)

In [ ]:
# size of feature set
print(X.shape)

In [ ]:
# define y
y = data.iloc[:, 12].values
y.shape

In [ ]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
# MODEL: SVM, linear
#will also go through each row as many times as you want it to - default is 5, making changes to weights, trying to
#move them closer and closer to lessen cost i think

from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

#getting only 25% of positives correctly classified 

In [ ]:
# MODEL: logistic regression
from sklearn import linear_model
#alpha, heavily penalizes the weights 
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

In [ ]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

In [ ]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()